In [1]:
import os

os.environ['JAVA_HOME'] = "C:/Program Files/Java/jdk-11"
os.environ['PYSPARK_PYTHON'] = "C:/Users/usr/anaconda3/envs/pyspark_env/python.exe"
os.environ['PYSPARK_DRIVER_PYTHON'] = "C:/Users/usr/anaconda3/envs/pyspark_env/python.exe"
os.environ['HADOOP_HOME'] = "C:/hadoop-3.4.0"
os.environ['HADOOP_COMMON_LIB_NATIVE_DIR'] = "C:/hadoop-3.4.0/lib/native"
os.environ['PATH'] += os.pathsep + "C:/hadoop-3.4.0/bin"

import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

- **Calcule la cantidad de películas en las que participó cada actor. La salida debe tener dos columnas: actor y conteo. La salida debe ordenarse por el conteo en orden descendente.**

In [12]:
movies = spark.read.option('sep', '|').option('inferSchema', 'True').option('header', 'True').csv('./data/movies.csv')
ratings = spark.read.option('sep', '|').option('inferSchema', 'True').option('header', 'True').csv('./data/movie_ratings.csv')

In [13]:
movies.show(truncate=False)

+-----------------+---------------------------+----+
|actor            |pelicula                   |año |
+-----------------+---------------------------+----+
|McClure, Marc (I)|Freaky Friday              |2003|
|McClure, Marc (I)|Coach Carter               |2005|
|McClure, Marc (I)|Superman II                |1980|
|McClure, Marc (I)|Apollo 13                  |1995|
|McClure, Marc (I)|Superman                   |1978|
|McClure, Marc (I)|Back to the Future         |1985|
|McClure, Marc (I)|Back to the Future Part III|1990|
|Cooper, Chris (I)|Me, Myself & Irene         |2000|
|Cooper, Chris (I)|October Sky                |1999|
|Cooper, Chris (I)|Capote                     |2005|
|Cooper, Chris (I)|The Bourne Supremacy       |2004|
|Cooper, Chris (I)|The Patriot                |2000|
|Cooper, Chris (I)|The Town                   |2010|
|Cooper, Chris (I)|Seabiscuit                 |2003|
|Cooper, Chris (I)|A Time to Kill             |1996|
|Cooper, Chris (I)|Where the Wild Things Are  

In [15]:
from pyspark.sql.functions import col, desc

In [17]:
movies.groupBy('actor').count().orderBy(desc('count')).show()

+-------------------+-----+
|              actor|count|
+-------------------+-----+
|   Tatasciore, Fred|   38|
|      Welker, Frank|   38|
| Jackson, Samuel L.|   32|
|      Harnell, Jess|   31|
|        Damon, Matt|   27|
|      Willis, Bruce|   27|
|  Cummings, Jim (I)|   26|
|         Hanks, Tom|   25|
|   Lynn, Sherry (I)|   25|
|    McGowan, Mickie|   25|
|    Bergen, Bob (I)|   25|
|      Proctor, Phil|   24|
|   Wilson, Owen (I)|   23|
|        Cruise, Tom|   23|
|         Pitt, Brad|   23|
|Freeman, Morgan (I)|   22|
|Williams, Robin (I)|   22|
|       Depp, Johnny|   22|
|     Morrison, Rana|   22|
|      Diaz, Cameron|   21|
+-------------------+-----+
only showing top 20 rows



- **Calcule la cantidad de películas producidas cada año. La salida debe tener tres columnas: año, siglo al que pertenece el año y conteo. La salida debe ordenarse por el conteo en orden descendente.**

In [19]:
from pyspark.sql.functions import countDistinct, when, lit

In [25]:
movies.groupBy('año').agg(
    countDistinct('pelicula').alias('conteo')
).withColumn('siglo', when((col('año') >= 1900) & (col('año') < 2000), lit('XX')).otherwise(lit('XXI'))).orderBy(desc('conteo')).select(
    col('año'),
    col('siglo'),
    col('conteo')
).show()

+----+-----+------+
| año|siglo|conteo|
+----+-----+------+
|2006|  XXI|    86|
|2004|  XXI|    86|
|2011|  XXI|    86|
|2005|  XXI|    85|
|2008|  XXI|    82|
|2002|  XXI|    81|
|2010|  XXI|    78|
|2000|  XXI|    77|
|2003|  XXI|    76|
|2007|  XXI|    75|
|2001|  XXI|    71|
|2009|  XXI|    68|
|1999|   XX|    67|
|1997|   XX|    66|
|1998|   XX|    59|
|1996|   XX|    42|
|2012|  XXI|    32|
|1995|   XX|    25|
|1994|   XX|    16|
|1986|   XX|    16|
+----+-----+------+
only showing top 20 rows



- **Obtenga la película con la calificación más alta por año. La salida debe tener solo una película por año y debe contener tres columnas: año, título de la película y valoración.**

In [26]:
ratings.show(truncate=False)

+----------+--------------------------+----+
|valoracion|pelicula                  |año |
+----------+--------------------------+----+
|1.6339    |'Crocodile' Dundee II     |1988|
|7.6177    |10                        |1979|
|1.2864    |10 Things I Hate About You|1999|
|0.3243    |10,000 BC                 |2008|
|0.3376    |101 Dalmatians            |1996|
|0.5218    |102 Dalmatians            |2000|
|12.8205   |1066                      |2012|
|0.6829    |12                        |2007|
|7.4061    |12 Rounds                 |2009|
|2.3677    |127 Hours                 |2010|
|1.3585    |13 Going on 30            |2004|
|8.4034    |13 game sayawng           |2006|
|0.59      |1408                      |2007|
|4.4292    |15 Minutes                |2001|
|2.2118    |16 Blocks                 |2006|
|1.0491    |17 Again                  |2009|
|3.9265    |1941                      |1979|
|10.4757   |2 Days in the Valley      |1996|
|0.4       |2 Fast 2 Furious          |2003|
|11.1111  

In [28]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, max

In [29]:
windowsSpec = Window.partitionBy('año').orderBy(desc('valoracion'))
windowsSpecAgg = Window.partitionBy('año')

In [32]:
ratings.withColumn('row_number', row_number().over(windowsSpec)).withColumn('max_val', max('valoracion').over(windowsSpecAgg)).filter(
    col('row_number') == 1
    ).select(
    col('año'),
    col('pelicula'),
    col('max_val')
).show()

+----+--------------------+-------+
| año|            pelicula|max_val|
+----+--------------------+-------+
|1937|Snow White and th...| 2.2207|
|1939|    The Wizard of Oz| 7.9215|
|1940|           Pinocchio| 7.8557|
|1942|               Bambi| 1.5053|
|1946|   Song of the South|  7.602|
|1950|          Cinderella| 9.4226|
|1953|           Peter Pan| 5.4756|
|1954|         Rear Window|10.7625|
|1955|  Lady and the Tramp| 5.1258|
|1956|Around the World ...|14.0607|
|1959|     Sleeping Beauty| 6.3919|
|1960|              Psycho|10.6375|
|1961|One Hundred and O...| 0.6726|
|1962|     The Longest Day|12.8866|
|1963|It's a Mad Mad Ma...|  6.626|
|1964|        My Fair Lady|  7.587|
|1965|      Doctor Zhivago| 4.9304|
|1966|Who's Afraid of V...|11.1111|
|1967|     The Dirty Dozen| 13.388|
|1968|        The Love Bug|13.4383|
+----+--------------------+-------+
only showing top 20 rows

